In [1]:
import os
import user_interaction
import normalize
import pandas as pd

In [ ]:
crime_df = pd.read_csv(os.path.join(os.getcwd(), 'crime.csv'))
crime_df = crime_df.drop(columns=crime_df.columns[10:])
crime_df = crime_df.dropna()
crime_df = crime_df.drop_duplicates()

In [2]:
pollution_df = pd.read_csv(os.path.join(os.getcwd(), 'pollution.csv'))
pollution_df = pollution_df.drop(columns=pollution_df.columns[9:])
pollution_df_small = pollution_df.drop(range(300000, 1746661))

In [ ]:
food_df = pd.read_csv(os.path.join(os.getcwd(), 'food.csv'), encoding='latin1')
food_df = food_df.drop(columns=food_df.columns[10:])

In [ ]:
liquor_df = pd.read_csv(os.path.join(os.getcwd(), 'liquor.csv'))
liquor_df = liquor_df.drop(columns=liquor_df.columns[12:])
liquor_df = liquor_df.drop(range(3000000, 12591077))
liquor_df = liquor_df.dropna()
liquor_df = liquor_df.drop_duplicates()

In [3]:
import time
start = time.time()
deps = user_interaction.find_dependencies(pollution_df_small)
time.time() - start

100%|██████████| 9/9 [00:31<00:00,  3.89s/it]


32.13678288459778

In [3]:
import time
start = time.time()
deps = user_interaction.find_dependencies(pollution_df_small)
time.time() - start

  0%|          | 0/9 [00:00<?, ?it/s]


UnboundLocalError: local variable 'attr' referenced before assignment

In [14]:
print(deps)

 --> Unnamed: 0
 {State}  {City}  --> State Code
 {State,City}  {State Code,City}  {Unnamed: 0,City}  {County}  {Address}  {Site Num,City}  --> County Code
 {City,County Code}  {Unnamed: 0,City}  {Address}  {County,City}  --> Site Num
 {Site Num,City}  {County,Site Num}  {Site Num,County Code}  --> Address
 {City}  --> State
 {City,County Code}  {Site Num,County Code}  {Address}  {State,County Code}  {State Code,County Code}  --> County
 {Address}  --> City
 --> Date Local


In [15]:
new_grps = normalize.normalize(deps)

In [16]:
for grp in new_grps:
    print(grp)
    print("\n\n")

 --> Unnamed: 0
 --> City
 --> Date Local



 --> City
 {City}  --> Address
 {City}  --> State



 --> State
 {State}  --> State Code



 --> Address
 {Address}  --> Site Num
 {Address}  --> County



 --> County
 {County}  --> County Code





### Food Example Compacted

In [ ]:
deps = user_interaction.find_dependencies(food_df, 1.00)
print(deps)
deps = user_interaction.find_dependencies(food_df, .95)
print(deps)

In [ ]:
auto_norm_dfs = user_interaction.auto_normalize(food_df)
for df in auto_norm_dfs:
    print(df)

deps = user_interaction.find_dependencies(food_df)
print(deps)
new_grps = normalize.normalize(deps)
for x in new_grps:
    print('\n ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ \n')
    print(x)

# Crime Example 

In [ ]:
auto_crime = user_interaction.auto_normalize(crime_df)

In [ ]:
crime_deps = user_interaction.find_dependencies(crime_df)
print(crime_deps)

In [ ]:
new_grps_crime = normalize.normalize(crime_deps)
for x in new_grps_crime:
    print('\n ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ \n')
    print(x)

# Liquor Example

In [ ]:
liquor_deps = user_interaction.find_dependencies(liquor_df)
print(liquor_deps)

In [ ]:
new_grps_liquor = normalize.normalize(liquor_deps)
for x in new_grps_liquor:
    print('\n ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~  \n')
    print(x)